<a href="https://colab.research.google.com/github/laysabelici/EstudosDeDataScience/blob/master/raizen_atualizacao_de_bases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Adicionar o pip
#!pip install os-sys
#!pip install numpy
#!pip install pandas
#!pip install urllib
#!pip install openpyxl
#!pip install requests_html
#!pip install urllib
!pip install requests-html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.5 MB/s 
     |████████████████████████████████| 139 kB 49.1 MB/s 
     |████████████████████████████████| 112 kB 35.8 MB/s 
     |████████████████████████████████| 127 kB 43.7 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=5fe4853c75305d63c268cc58328ca1e9ebf993b3ef61bcce55a1a7a04d6136be
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=7eb4b4584689228aa8e9844288592b5e2a9409f7628807b157d0cca165f3bcb5
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
Successfully built fake-useragent parse
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

Importando as bibliotecas necessárias

In [4]:
import os
import numpy as np
import pandas as pd
import urllib.request
import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from requests_html import HTMLSession
import urllib

### Obtenção dos dados



Criando o diretório para armazenamento das bases de dados

In [5]:
diretorio = './bases_diarias'

try:
    os.makedirs(diretorio)
    print('Diretório gerado com sucesso.')
except:
    print('Esse diretório já existe.')

Diretório gerado com sucesso.


Definido a url de dowload das bases de dados originais

In [6]:
datasets = {
    'dataset_derivados': 'https://dados.gov.br/dataset/vendas-de-derivados-de-petroleo-e-biocombustiveis/resource/c3d6e0b4-f86e-48f8-9325-6cc0d434b33f?inner_span=True',
    'dataset_diesel': 'https://dados.gov.br/dataset/vendas-de-derivados-de-petroleo-e-biocombustiveis/resource/2429fdeb-df86-4e63-b248-2038f6c3e3cc?inner_span=True'
}

Acessando a url do repositório de dados para pegar a última versão disponível dos dados (caso uma nova versão esteja disponível as urls serão atualizadas)

In [7]:
try:
  for dataset in datasets:
    hiddens = HTMLSession().get(datasets[dataset]).html.find('a', containing='https://www.gov.br/anp/pt-br/centrais-de-conteudo', first=True)
    datasets[dataset] = hiddens.links
    print(f'Base de dados {dataset} atualizada com sucesso!')
except:
  print(f'Base de dados {dataset} não foi atualizada!')

Base de dados dataset_derivados atualizada com sucesso!
Base de dados dataset_diesel atualizada com sucesso!


Realizando o download das bases de dados

In [8]:
try:
  for dataset in datasets:
    url = str(datasets[dataset]).strip("{'}")
    urllib.request.urlretrieve(url, f'./bases_diarias/dados_{dataset}.csv')
    print(f"Base de dados {dataset} atualizada.")
except:
    print(f"Não foi possível fazr o download da base de dados {dataset}.")  

Base de dados dataset_derivados atualizada.
Base de dados dataset_diesel atualizada.


In [9]:
urllib.request.urlretrieve("https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-estatisticos/de/vdpb/vendas-combustiveis-m3.xls/@@download/file/vendas-combustiveis-m3.xlsx", "./bases_diarias/vendas-combustiveis-m3.xlsx")
print("Dados pivotados baixados.") 

Dados pivotados baixados.


### Tratamento dos dados

Obtenção dos dados gerais de vendas de combustíveis




In [10]:
dados_gerais_vendas_combustiveis = openpyxl.load_workbook('./bases_diarias/vendas-combustiveis-m3.xlsx')
tabela_vendas_combustiveis = dados_gerais_vendas_combustiveis['Plan1']

Obtenção da data de atualização dos dados

In [11]:
data_atualizacao_dados = [tabela_vendas_combustiveis.cell(row=42, column=2).value]

print(data_atualizacao_dados)

['Dados atualizados em 282 de julho de 2022.']


Armazenar no dataframe a data de atualização dos dados

In [12]:
dataframe_vendas_combustiveis = pd.DataFrame(data_atualizacao_dados, columns =['at_data'])

print(dataframe_vendas_combustiveis)

                                      at_data
0  Dados atualizados em 282 de julho de 2022.


Criando máscara de formatação do dataframe

Carregando a base de dados de derivados e alterando o nome das colunas

In [13]:
dataframe_derivados = pd.read_csv('./bases_diarias/dados_dataset_derivados.csv', sep = ';');
dataframe_derivados = dataframe_derivados.rename(columns = {'UNIDADE DA FEDERAÇÃO': 'uf', 'GRANDE REGIÃO': 'region', 'PRODUTO': 'product', 'MÊS': 'month', 'ANO': 'year', 'VENDAS': 'volume'})

print(dataframe_derivados)

       year month           region            uf           product      volume
0      1999   NOV   REGIÃO SUDESTE  MINAS GERAIS  ETANOL HIDRATADO   43356,352
1      1999   FEV   REGIÃO SUDESTE     SÃO PAULO  ETANOL HIDRATADO  202406,862
2      1999   SET  REGIÃO NORDESTE       SERGIPE  ETANOL HIDRATADO    2744,883
3      1999   AGO  REGIÃO NORDESTE       SERGIPE  ETANOL HIDRATADO    2661,218
4      1999   MAI   REGIÃO SUDESTE     SÃO PAULO  ETANOL HIDRATADO  249336,184
...     ...   ...              ...           ...               ...         ...
84235  2001   MAI     REGIÃO NORTE          ACRE       ÓLEO DIESEL   20051,914
84236  2001   MAR     REGIÃO NORTE      RONDÔNIA       ÓLEO DIESEL   38061,666
84237  2001   SET     REGIÃO NORTE      RONDÔNIA       ÓLEO DIESEL   58384,106
84238  2001   FEV     REGIÃO NORTE      RONDÔNIA       ÓLEO DIESEL   30995,293
84239  2001   JAN     REGIÃO NORTE      RONDÔNIA       ÓLEO DIESEL   42405,269

[84240 rows x 6 columns]


Selecionando colunas do dataframe de derivados a serem utilizadas na análise

In [14]:
dataframe_derivados = dataframe_derivados[['product', 'year', 'uf', 'month']]
print(dataframe_derivados)

                product  year            uf month
0      ETANOL HIDRATADO  1999  MINAS GERAIS   NOV
1      ETANOL HIDRATADO  1999     SÃO PAULO   FEV
2      ETANOL HIDRATADO  1999       SERGIPE   SET
3      ETANOL HIDRATADO  1999       SERGIPE   AGO
4      ETANOL HIDRATADO  1999     SÃO PAULO   MAI
...                 ...   ...           ...   ...
84235       ÓLEO DIESEL  2001          ACRE   MAI
84236       ÓLEO DIESEL  2001      RONDÔNIA   MAR
84237       ÓLEO DIESEL  2001      RONDÔNIA   SET
84238       ÓLEO DIESEL  2001      RONDÔNIA   FEV
84239       ÓLEO DIESEL  2001      RONDÔNIA   JAN

[84240 rows x 4 columns]


In [15]:
dataframe_derivados['month'] = dataframe_derivados['month'].map({'JAN':'1',
                                                                 'FEV' : '2',
                                                                 'MAR' : '3',
                                                                 'ABR' : '4',
                                                                 'MAI' : '5',
                                                                 'JUN' : '6',
                                                                 'JUL' : '7',
                                                                 'AGO' : '8',
                                                                 'SET' : '9',
                                                                 'OUT' : '10',
                                                                 'NOV' : '11',
                                                                 'DEZ' : '12'},
                                                                 na_action=None)

print(dataframe_derivados)

                product  year            uf month
0      ETANOL HIDRATADO  1999  MINAS GERAIS    11
1      ETANOL HIDRATADO  1999     SÃO PAULO     2
2      ETANOL HIDRATADO  1999       SERGIPE     9
3      ETANOL HIDRATADO  1999       SERGIPE     8
4      ETANOL HIDRATADO  1999     SÃO PAULO     5
...                 ...   ...           ...   ...
84235       ÓLEO DIESEL  2001          ACRE     5
84236       ÓLEO DIESEL  2001      RONDÔNIA     3
84237       ÓLEO DIESEL  2001      RONDÔNIA     9
84238       ÓLEO DIESEL  2001      RONDÔNIA     2
84239       ÓLEO DIESEL  2001      RONDÔNIA     1

[84240 rows x 4 columns]


Carregando a base de dados de diesel e alterando o nome das colunas

In [16]:
dataframe_diesel =pd.read_csv('./bases_diarias/dados_dataset_diesel.csv', sep = ';')
dataframe_diesel = dataframe_diesel.rename(columns = { 'UNIDADE DA FEDERAÇÃO': 'uf', 'PRODUTO': 'product', 'MÊS':'month', 'ANO':'year', 'VENDAS': 'volume'})

print(dataframe_diesel)

       year month        GRANDE REGIÃO                uf  \
0      2013   JAN         REGIÃO NORTE          RONDÔNIA   
1      2013   FEV         REGIÃO NORTE          RONDÔNIA   
2      2013   MAR         REGIÃO NORTE          RONDÔNIA   
3      2013   ABR         REGIÃO NORTE          RONDÔNIA   
4      2013   MAI         REGIÃO NORTE          RONDÔNIA   
...     ...   ...                  ...               ...   
15385  2022   FEV  REGIÃO CENTRO-OESTE  DISTRITO FEDERAL   
15386  2022   MAR  REGIÃO CENTRO-OESTE  DISTRITO FEDERAL   
15387  2022   ABR  REGIÃO CENTRO-OESTE  DISTRITO FEDERAL   
15388  2022   MAI  REGIÃO CENTRO-OESTE  DISTRITO FEDERAL   
15389  2022   JUN  REGIÃO CENTRO-OESTE  DISTRITO FEDERAL   

                     product   volume  
0           ÓLEO DIESEL S-10   3517,6  
1           ÓLEO DIESEL S-10   3681,7  
2           ÓLEO DIESEL S-10  4700,67  
3           ÓLEO DIESEL S-10   5339,2  
4           ÓLEO DIESEL S-10   6166,4  
...                      ...      ...  

Selecionando colunas do dataframe de diesel a serem utilizadas na análise

In [17]:
dataframe_diesel = dataframe_diesel[['product', 'year', 'uf', 'month']]

print(dataframe_diesel)

                     product  year                uf month
0           ÓLEO DIESEL S-10  2013          RONDÔNIA   JAN
1           ÓLEO DIESEL S-10  2013          RONDÔNIA   FEV
2           ÓLEO DIESEL S-10  2013          RONDÔNIA   MAR
3           ÓLEO DIESEL S-10  2013          RONDÔNIA   ABR
4           ÓLEO DIESEL S-10  2013          RONDÔNIA   MAI
...                      ...   ...               ...   ...
15385  ÓLEO DIESEL (OUTROS )  2022  DISTRITO FEDERAL   FEV
15386  ÓLEO DIESEL (OUTROS )  2022  DISTRITO FEDERAL   MAR
15387  ÓLEO DIESEL (OUTROS )  2022  DISTRITO FEDERAL   ABR
15388  ÓLEO DIESEL (OUTROS )  2022  DISTRITO FEDERAL   MAI
15389  ÓLEO DIESEL (OUTROS )  2022  DISTRITO FEDERAL   JUN

[15390 rows x 4 columns]


In [18]:
dataframe_derivados['month'] = dataframe_derivados['month'].map({'JAN':'1',
                                                                 'FEV' : '2',
                                                                 'MAR' : '3',
                                                                 'ABR' : '4',
                                                                 'MAI' : '5',
                                                                 'JUN' : '6',
                                                                 'JUL' : '7',
                                                                 'AGO' : '8',
                                                                 'SET' : '9',
                                                                 'OUT' : '10',
                                                                 'NOV' : '11',
                                                                 'DEZ' : '12'},
                                                                 na_action=None)

print(dataframe_derivados)

                product  year            uf month
0      ETANOL HIDRATADO  1999  MINAS GERAIS   NaN
1      ETANOL HIDRATADO  1999     SÃO PAULO   NaN
2      ETANOL HIDRATADO  1999       SERGIPE   NaN
3      ETANOL HIDRATADO  1999       SERGIPE   NaN
4      ETANOL HIDRATADO  1999     SÃO PAULO   NaN
...                 ...   ...           ...   ...
84235       ÓLEO DIESEL  2001          ACRE   NaN
84236       ÓLEO DIESEL  2001      RONDÔNIA   NaN
84237       ÓLEO DIESEL  2001      RONDÔNIA   NaN
84238       ÓLEO DIESEL  2001      RONDÔNIA   NaN
84239       ÓLEO DIESEL  2001      RONDÔNIA   NaN

[84240 rows x 4 columns]
